In [2]:
import pandas as pd

In [15]:
train_csv = pd.read_csv("data/sign_data/train_data.csv", header=None)
test_csv = pd.read_csv("data/sign_data/test_data.csv", header=None)
train_csv.columns = ["id", "path", "forg"]
test_csv.columns = ["id", "path", "forg"]

In [17]:
train_csv.head()

,id,path,forg
0,068/09_068.png,068_forg/03_0113068.PNG,1
1,068/09_068.png,068_forg/01_0124068.PNG,1
2,068/09_068.png,068_forg/02_0124068.PNG,1
3,068/09_068.png,068_forg/01_0113068.PNG,1
4,068/09_068.png,068_forg/04_0124068.PNG,1
